In [ ]:
#Standard inmorts 
import numpy as np
import pandas as pd
import os
#import feather
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
#import pysubgroup as ps

from scipy import io
from scipy import stats



#the rest should be ML 

from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# K-Means 
from sklearn.cluster import KMeans


# Pipeline stuff
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from scipy.cluster.hierarchy import dendrogram
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

#K-Prototypes
from kmodes.kprototypes import KPrototypes

#metrics
import sklearn.metrics as metrics
from sklearn.metrics import pairwise_distances

## settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 1. Read Data

In [ ]:
df1 = pd.read_feather('../Data3/D3_MKPF-MSEG_raw.ftr', columns=None, use_threads=True)
df2 = pd.read_feather('../Data3/D3_MKPF-MSEG_cleaned.ftr', columns=None, use_threads=True)

### Data inspection

In [ ]:
df1.head(2)

In [ ]:
df2.sample(2)

In [ ]:
# corretion datatype
df2['BSTMG'] = df2['BSTMG'].astype(int)

In [ ]:
df2['BSTMG'].dtype

# FILTERING Data for PM

In [ ]:
# 601 = WA zu Kunde -> NB im P2P
#df_WA = df2.loc[(df2['BWART'] == '601')]
# 301 = Umlagerung -> NB im P2P
#df_UL = df2.loc[(df2['BWART'] == '301')]
# 261 = Verbrauch an Fert -> NB im P2P
#df_VB = df2.loc[(df2['BWART'] == '261')]

# 101 = WE Roh
df3 = df2.loc[((df2['BWART'] == '101') &  (df2['LGORT'] == '88'))].copy()

In [ ]:
print(df2.shape)
print(df3.shape)

In [ ]:
df3.head(2)

In [ ]:
df3 = df3.reset_index(drop=True).copy()

### Del Data before clustering

We won't use some data that is used in PM for the clustering, hence it would drastically change 

an therefore create final dataset used

In [ ]:
df = df3.drop(['CPUDT', 'TMSTMP', 'CPUTM', 'MBLNR', 'EBELN','EBELP', 'XBLNR_MKPF'], axis=1).copy()

*Additional cleansing hence the new Dataset as subset has informational waste*

In [ ]:
df.shape

In [ ]:
df.head()

### Additional Cleaning

In [ ]:
# Spalten die nur NAN enthalten entfernen
df = df.dropna(axis=1, how='all')


# Spalten die ausschließlich XY Werte enthalten (0 / X / 1 / etc.) dropen
df = df.loc[:, (df.astype(str) != 0).any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,00').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,000').any(axis=0)]
df = df.loc[:, (df.astype(str) != 'X').any(axis=0)]
df = df.loc[:, (df.astype(str) != '1').any(axis=0)]



#spaletn entfernen die immer die selben Werte enthalten
#nunique = df.apply(pd.Series.nunique)
#cols_to_drop = nunique[nunique == 1].index
#df = df.drop(cols_to_drop, axis=1)
#df.drop(columns=df.columns[df.nunique()==1], inplace=True)

# keep_columns = [col for col in df.columns if len(df[col].unique()) > 1]
# df = df[keep_columns].copy()


#print(len(df_io.index))
print(df.shape)

In [ ]:
col_cat = list(df.select_dtypes(include=['category']).columns)
col_num = list(df.select_dtypes(include=['float64', 'int64']).columns)

In [ ]:
print(len(col_cat))
print('plus')
print(len(col_num))
print('ergibt:')
print(df.shape[1])

# 2. Clustering using K-Prototype

Approach: 
- analyse data to set number of clusters
    - test 

In [ ]:
df_kp = df.copy()

In [ ]:
#df_kp.dtypes

In [ ]:
df_corc = df_kp.copy()

In [ ]:
# Change dtype to Category for relevant features
df_corc = df_kp.copy()

# change Dtype to Cat for categorical columns
cats_cat = df_corc[col_cat]
        
for (columnName, columnData) in cats_cat.items():
    df_corc[columnName] = df_corc[columnName].astype(str)
    if isinstance(df_corc[columnName][0], str):
        s = pd.Series(df_corc[columnName], dtype="category")
        df_corc[columnName] = s.cat.rename_categories(np.arange(len(np.unique(df_corc[columnName]))))

In [ ]:
df_corc.head()

In [ ]:
df_corc.shape

## GAP Statistics with K-Proto

In [ ]:
# GAP
import fg_cleansing_helper_new as fg


#create a K-Proto Classifier
catColumnsPos = [df_corc.columns.get_loc(col) for col in list(df_corc.select_dtypes('category').columns)]
kprototype = KPrototypes(n_jobs = -1, init = 'Cao', random_state = 0, max_iter = 100, verbose=False)

#Compute gap with Classifier
k_max = 10
gap, reference_inertia, ondata_inertia = fg.compute_gap_k_proto(kprototype, df_corc.to_numpy(), catColumnsPos, k_max)

# plot results 1
plt.plot(range(1, k_max+1), reference_inertia,
         '-+', label='reference')
plt.plot(range(1, k_max+1), ondata_inertia,
         '-+', label='data')
plt.xlabel('k')
plt.ylabel('log(inertia)')
plt.show()

In [ ]:
# plot GAP Statistics
plt.plot(range(1, k_max+1), gap, '-o')
plt.ylabel('gap')
plt.xlabel('k')

# 2.2 CLustering K-Proto

In [ ]:

random_state = 1234
pd.options.plotting.backend = "plotly"
#import timeit

# Convert dataframe to matrix
dfMatrix = df_corc.to_numpy()
# Choose optimal K using Elbow method
l_elbow = []
l_silh = []
l_chi1 = []
l_dbsc = []
print(dfMatrix.shape)
print(df_corc.shape)

#define cat vars for K-prototypes as numerical list
catColumnsPos = [df_corc.columns.get_loc(col) for col in list(df_corc.select_dtypes('category').columns)]
print(catColumnsPos)
print('Categorical columns: {}'.format(list(df_corc.select_dtypes('category').columns)))


for cluster in range(1, 20):
    try:
        kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Cao', random_state = 0, max_iter = 100, verbose=False) #Cao
        #print('step')
        kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)
        print('cluster:' + str(cluster))
        l_elbow.append(kprototype.cost_) #inertia_
        

        print('Cluster initiation: {}'.format(cluster))
        #lets calc some scores
        if cluster > 1:
            l_chi1.append(metrics.calinski_harabasz_score(dfMatrix, kprototype.labels_))
            l_silh.append(metrics.silhouette_score(dfMatrix, kprototype.labels_, metric='euclidean'))
            l_dbsc.append(metrics.davies_bouldin_score(dfMatrix, kprototype.labels_))
        #test
        else:
            l_dbsc.append(0)
            l_chi1.append(0)
            l_silh.append(0)

    except:
        break
    

print('finished Kproto')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


mypal2 =['#FEAEFE', '#FC05FB', '#B4FFE4', '#3FFEBA', '#1F363D', '#2B4570', '#3A2D32', '#706F6F']

a = len(l_dbsc) + 1

df_chi = pd.DataFrame({'Cluster':range(1, a), 'CH value':l_chi1})
df_silh = pd.DataFrame({'Cluster':range(1, a), 'S Score':l_silh})
df_elbow = pd.DataFrame({'Cluster':range(1, a), 'Cost':l_elbow})
df_dbsc = pd.DataFrame({'Cluster':range(1, a), 'DB value':l_dbsc})

fig = make_subplots(rows=1, cols=4) #for titles above -->
subplot_titles=("CH value", "Silhouette index", "Elbow", 'DB value')

trace1 = px.line(df_chi, x="Cluster", y="CH value")
trace2 = px.line(df_silh, x="Cluster", y="S Score")
trace3 = px.line(df_elbow, x="Cluster", y="Cost")
trace4 = px.line(df_dbsc, x="Cluster", y="DB value")

#1
fig.add_trace(go.Scatter(mode='lines+markers', x=trace1.data[0]['x'], y=trace1.data[0]['y'],text=trace1.data[0]['x'], line = dict(color='#FEAEFE', width=3), marker=dict(size=8), legendgroup="Calinski Harabasz Index"), row=1, col=1, ) #, name="Chi"
#fig.update_traces(textposition='top right', line = dict(color='#FEAEFE', width=3), marker=dict(size=8))
fig['layout']['yaxis1']['title']='CH value'
fig['layout']['yaxis2']['title']='Silhouette Score'
fig['layout']['yaxis3']['title']='Elbow'
fig['layout']['yaxis4']['title']='DB Value'


#2
fig.add_trace(go.Scatter(mode='lines+markers', x=trace2.data[0]['x'], y=trace2.data[0]['y'], text=trace2.data[0]['x'], line = dict(color='#00F19D', width=3), marker=dict(size=8), legendgroup="Sillhouette Score"), row=1, col=2) #, name="Silhouette"
#fig.update_traces(textposition='top right')

#3
fig.add_trace(go.Scatter(mode='lines+markers', x=trace3.data[0]['x'], y=trace3.data[0]['y'], text=trace3.data[0]['x'], line = dict(color='#FC05FB', width=3), marker=dict(size=8), legendgroup="Elbow Method"), row=1, col=3) #, name="Elbow"
#fig.update_traces(textposition='top right')

#3
fig.add_trace(go.Scatter(mode='lines+markers', x=trace4.data[0]['x'], y=trace4.data[0]['y'], text=trace4.data[0]['x'], line = dict(color='#1F363D', width=3), marker=dict(size=8), legendgroup="DB value"), row=1, col=4) #, name="Elbow"
#fig.update_traces(textposition='top right')

# customization
fig.update_traces(textposition='top right')
#fig['layout']['xaxis1']['title']='CH value'
#fig['layout']['xaxis2']['title']='Silhouette index'
#fig['layout']['xaxis3']['title']='Elbow'

fig.update_xaxes(nticks=10, linecolor='#F5F5F5', gridcolor='#F5F5F5')
fig.update_yaxes(linecolor='#F5F5F5', gridcolor='#F5F5F5')
fig.update_layout(plot_bgcolor='white')
fig.update_layout(height=290, width=1200)
fig.update_layout(margin=dict(l=10, r=10, t=30, b=20))
#fig.update_layout(title_text="Clustering Metrics for K-Prototypes")
#fig.update_layout(showlegend=False)
fig.update_layout(legend=dict(
    orientation="h",
    xanchor="right",
    yanchor="bottom",
    y=-0.3, 
    x=1 
))
fig.update_layout(showlegend=False)
fig.show()

#export
fig.write_image("Metrics_placeholder.pdf")

In [ ]:
dfMatrix2 = df_kp.copy()

# number of clusters
K = 2
# no. of points
N = df_corc.shape[0] 
# no. of dimensions
M = df_corc.shape[1]

# no. of numerical dimensions
#MN = 5
# no. of times test is repeated
#T = 3

kp = KPrototypes(n_clusters=K, init='Huang', n_init=1, verbose=True)
kp.fit_predict(dfMatrix2, categorical = catColumnsPos)

print(kp.cluster_centroids_)
print(kp.labels_)
print(kp.labels_.shape)

In [ ]:
#add results to original Dataset 
df3['kproto_labels'] = kp.labels_

In [ ]:
#kp.labels_

## Cluster Metrics
- elbwo
- silloutte 
- gap statistics
- harabaz index


In [ ]:
print(metrics.silhouette_score(dfMatrix, kp.labels_, metric='euclidean'))
print(metrics.calinski_harabasz_score(dfMatrix, kp.labels_))
print(metrics.davies_bouldin_score(dfMatrix, kp.labels_))
print(np.unique(kp.labels_, return_counts=True))

In [ ]:
df3.sample(5)

# 3. clustering with DBSCAN

In [ ]:
df_kp.head(2)

In [ ]:
# df_gw = df_kp.iloc[:,:].copy()
df_gw = df_kp.copy()

In [ ]:
df_gw.shape
#df_corc.head(5) alternative


In [ ]:
import gower
distance_matrix = gower.gower_matrix(np.asarray(df_gw))

In [ ]:
distance_matrix.shape

## 3.1 initial DBSCAN testwith wide range cluster

In [ ]:
df_kp.shape

In [ ]:
from sklearn.cluster import DBSCAN

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.15, 
                        min_samples=25, 
                        metric="precomputed")

# Fitting the clustering algorithm
dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
#df3["dbscan_gower"] = dbscan_cluster.labels_

In [ ]:
dbscan_cluster.labels_.shape

In [ ]:
print(metrics.silhouette_score(distance_matrix, dbscan_cluster.labels_, metric='precomputed')) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, dbscan_cluster.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, dbscan_cluster.labels_)) #lower ist better 
print(np.unique(dbscan_cluster.labels_, return_counts=True))

## DBSCAN with calculated Eps

- min samples should be arround No. of features
- Eps can be calculated



In [ ]:
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt

neighbors = NearestNeighbors(n_neighbors=25, metric='precomputed')
neighbors_fit = neighbors.fit(distance_matrix)
distances, indices = neighbors_fit.kneighbors(distance_matrix)

distances = np.sort(distances, axis=0)
distances = distances[:,1]
#plt.plot(distances)

In [ ]:
import plotly.express as px


#mypal2 =['#FEAEFE', '#FC05FB', '#B4FFE4', '#3FFEBA', '#1F363D', '#2B4570', '#3A2D32', '#706F6F']
fig = px.line(distances, 
              width=500, height=500
              )
#linecolor
fig.update_traces(textposition='top right', line = dict(color='#FEAEFE', width=3), marker=dict(size=8))
#axes
fig.update_xaxes(nticks=10, linecolor='#F5F5F5', gridcolor='#F5F5F5')
fig.update_yaxes(linecolor='#F5F5F5', gridcolor='#F5F5F5')
#background
fig.update_layout(plot_bgcolor='white')
#show
fig.show()

In [ ]:
from sklearn.cluster import DBSCAN

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.05, 
                        min_samples=14, 
                        metric="precomputed")

# Fitting the clustering algorithm
dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
#df3["dbscan_gower"] = dbscan_cluster.labels_

In [ ]:
print(metrics.silhouette_score(distance_matrix, dbscan_cluster.labels_, metric='precomputed')) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, dbscan_cluster.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, dbscan_cluster.labels_)) #lower ist better 
print(np.unique(dbscan_cluster.labels_, return_counts=True))

In [ ]:
# Adding the last results to a new column in the dataframe
df3["dbscan_gower"] = dbscan_cluster.labels_

In [ ]:
#df3.sample(15)

In [ ]:
df3.loc[(df3.dbscan_gower == -1)].shape

In [ ]:
df3.loc[(df3.TCODE2_MKPF == 'MIGO_GR')].shape

In [ ]:
df3.sample(5)

##  4. Clustering with Aglomerative Clustering

In [ ]:
# Clustering algorithm
from sklearn.cluster import AgglomerativeClustering

# Rand Index
from sklearn.metrics.cluster import rand_score

# Encode labels
from sklearn import preprocessing

# Confusion Matrix
from sklearn.metrics import confusion_matrix

In [ ]:
from scipy.cluster import hierarchy

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    mypal2 =['#FEAEFE', '#FC05FB', '#B4FFE4', '#3FFEBA', '#1F363D', '#2B4570', '3A2D32', '706F6F']
    hierarchy.set_link_color_palette(['#FEAEFE', '#FC05FB', '#3FFEBA', '#2B4570'])
    dendrogram(linkage_matrix, **kwargs, above_threshold_color='#2b4570')

In [ ]:
K

### Single Linkage

In [ ]:
## linkage: single
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='single', metric='precomputed')

model = model.fit(distance_matrix)
plt.figure(figsize=(8,6))
plt.title("Agglomerative Clustering (single) Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
#test 
model_single = AgglomerativeClustering(n_clusters=K, linkage='single', metric='precomputed')
clusters_single = model_single.fit_predict(distance_matrix)

In [ ]:
print(metrics.silhouette_score(distance_matrix, model_single.labels_, metric="precomputed")) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, model_single.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, model_single.labels_)) #lower ist better
print(np.unique(clusters_single, return_counts=True))

In [ ]:
# Adding the results to a new column in the dataframe
df3["AgloSngl_gower"] = clusters_single

### Average Linkage

In [ ]:
## linkage: average
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='average', metric='precomputed')

model = model.fit(distance_matrix)
plt.figure(figsize=(8,6))
plt.title("Agglomerative Clustering (average) Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
model_average = AgglomerativeClustering(n_clusters=3, linkage='average', metric='precomputed')
clusters_average = model_average.fit_predict(distance_matrix)

In [ ]:
print(metrics.silhouette_score(distance_matrix, model_average.labels_, metric="precomputed")) #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, model_average.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, model_average.labels_)) #lower ist better 
print(np.unique(clusters_average, return_counts=True))

In [ ]:
# Adding the results to a new column in the dataframe
df3["AgloAvg_gower"] = clusters_average

### Complete Linkage

In [ ]:
## linkage: complete
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='complete', metric='precomputed')

model = model.fit(distance_matrix)
plt.figure(figsize=(8,6))
plt.title("Agglomerative Clustering (complete) Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.savefig("AgglomerativeClusteringDendrogram_complete.pdf")
plt.show()


In [ ]:
model_complete = AgglomerativeClustering(n_clusters=3, linkage='complete', metric='precomputed')
clusters_complete = model_complete.fit_predict(distance_matrix)

In [ ]:
print(metrics.silhouette_score(distance_matrix, model_complete.labels_, metric="precomputed"))  #> 0.5 should be good
print(metrics.calinski_harabasz_score(distance_matrix, model_complete.labels_)) #higher ist better 
print(metrics.davies_bouldin_score(distance_matrix, model_complete.labels_)) #lower ist better
print(np.unique(clusters_complete, return_counts=True))

In [ ]:
# Adding the results to a new column in the dataframe
df3["AgloComp_gower"] = clusters_complete

In [ ]:
df3.sample(10)

In [ ]:
df3.loc[df3.AgloComp_gower==2]

# EXPORT

In [ ]:
df3.head(2)

In [ ]:
df3.shape

In [ ]:
df3.to_feather('../Data3/D3_MB_XS_CL.ftr')